In [ ]:
# cargamos un fichero con trazas, usando código de t-mon

%run ProcessxAPISGStatement.py
%run fileBrowserAndUploadButtonToLoadProcessStatements.py
global players_info, df
out, err=[],[]
df={}
players_info={}
load_players_info_from_file("xapi-sg-sample-data.json", players_info, out, err)

In [ ]:
# cargamos pandas, plotly

import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
# montemos un dataframe
df = pd.DataFrame.from_dict(players_info, 'index')
df.loc['John Smith']

In [ ]:
# algo de información básica
print(f"total de {len(df)} actores")

### Intentemos ver puntuación en cada completable...

In [ ]:
# montamos un dataframe para valores de este gráfico en concreto
cvalues = []
for id, row in df[['completables_scores']].iterrows():
    for k, v in row['completables_scores'].items():
        cvalues.append({'id': id, 'completable': k, 'score': v})
cvalues = pd.DataFrame.from_records(cvalues)

In [ ]:
# mostramos
data = []
for c in cvalues['completable'].unique():
    bar_data = cvalues[cvalues['completable'] == c]
    data.append(go.Bar(x=bar_data['id'], y=bar_data['score'], name=c))    
fig = go.Figure(
    layout_title_text="Puntuación en completables",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='stack')
fig.show()

### Y ahora, veamos progreso en el tiempo, con y sin alinear al inicio del experimento

In [ ]:
# montamos un dataframe para valores de este gráfico en concreto
tvalues = []
for id, row in df[['completables_progress']].iterrows():
    for v in row['completables_progress']['MyFirstGame']:
        tvalues.append({'id': id, 'completable': 'MyFirstGame', 'progress': v[0], 'time': np.datetime64(v[1])})
tvalues = pd.DataFrame.from_records(tvalues)
tvalues

In [ ]:
# mostramos
data = []
for id in tvalues['id'].unique():
    bar_data = tvalues[tvalues['id'] == id]
    data.append(go.Scatter(x=bar_data['time'], y=bar_data['progress'], name=id, hovertext=f"<b>{id}</b>", mode="lines+markers"))    
fig = go.Figure(
    layout_title_text="Progreso en 1er completable",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
# y ahora, alineando por 1er evento
data = []
for id in tvalues['id'].unique():
    bar_data = tvalues[tvalues['id'] == id].copy()
    first = bar_data.time.min()
    bar_data.time = pd.to_timedelta(bar_data.time - first) + pd.to_datetime('1970/01/01')
    data.append(go.Scatter(x=bar_data['time'], y=bar_data['progress'], name=id, hovertext=f"<b>{id}</b>", mode="lines+markers"))    
fig = go.Figure(
    layout_title_text="Progreso en 1er completable, desde inicio de sesión",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.show()